# Events Stream Example

Minimal example showing the events API. Two cells:
1. Setup: Create agent, session, send message
2. View all events

Prerequisites: API server at `http://localhost:9000`

In [ ]:
import requests, time, json

API = "http://localhost:9000/v1"

# Create agent
agent = requests.post(f"{API}/agents", json={
    "name": "Events Demo",
    "system_prompt": "You are a helpful assistant. Be very brief."
}).json()
agent_id = agent["id"]
print(f"Agent: {agent_id}")

# Create session
session = requests.post(f"{API}/agents/{agent_id}/sessions", json={}).json()
session_id = session["id"]
print(f"Session: {session_id}")

# Send message (triggers agentic loop)
msg = requests.post(f"{API}/agents/{agent_id}/sessions/{session_id}/messages", json={
    "message": {"content": [{"type": "text", "text": "What is 2+2?"}]}
}).json()
print(f"Message sent: {msg['id']}")

In [ ]:
# Poll for events continuously until session completes
seen = set()
timeout = 30
start = time.time()

print("Polling for events...\n")

while time.time() - start < timeout:
    events = requests.get(f"{API}/agents/{agent_id}/sessions/{session_id}/events").json()
    
    for e in events["data"]:
        seq = e["sequence"]
        if seq in seen:
            continue
        seen.add(seq)
        
        # Print full event
        print(f"[{seq}] {e['event_type']}")
        print(json.dumps(e["data"], indent=2))
        print()
        
        # Stop polling when we get agent response
        if e["event_type"] == "message.agent":
            print("=" * 40)
            print("Agent responded. Done polling.")
            break
    else:
        time.sleep(0.5)
        continue
    break

In [ ]:
# Cleanup
requests.delete(f"{API}/agents/{agent_id}/sessions/{session_id}")
requests.delete(f"{API}/agents/{agent_id}")
print("Cleaned up agent and session")